# **Stable Diffusion WebUI + LoRA-Scipts训练包**

基础环境：modelscope:1.13.3-pytorch2.1.2tensorflow2.14.0-gpu-py310-cu121-ubuntu22.04
Python:3.10.12
CUDA: 12.1
Pytorch: 2.1.2+cu121
显卡: NVIDIA A10


## 一、 初始环境

In [ ]:
#只需运行一次
#查看是什么GPU
!nvidia-smi
#安装解压软件和下载存储工具
!apt-get update  > /dev/null 2>&1 
!apt-get install zip -y > /dev/null 2>&1
!apt install -y aria2 > /dev/null 2>&1 && echo 全部成功

In [ ]:
# 检查显卡
import torch
print('CUDA版本:',torch.version.cuda)
print('Pytorch版本:',torch.__version__)
print('显卡是否可用:','可用' if(torch.cuda.is_available()) else '不可用')
print('显卡数量:',torch.cuda.device_count())
print('是否支持BF16数字格式:','支持' if (torch.cuda.is_bf16_supported()) else '不支持不能训练sdxl')
print('当前显卡型号:',torch.cuda.get_device_name())
print('当前显卡的CUDA算力:',torch.cuda.get_device_capability())
print('当前显卡的总显存:',torch.cuda.get_device_properties(0).total_memory/1024/1024/1024,'GB')
print('是否支持TensorCore:','支持' if (torch.cuda.get_device_properties(0).major >= 7) else '不支持')
print('当前显卡的显存使用率:',torch.cuda.memory_allocated(0)/torch.cuda.get_device_properties(0).total_memory*100,'%')

In [ ]:
# 查看剩余内存空间
import subprocess
# 执行命令并解析输出
command = "df -h /mnt/workspace"
result = subprocess.check_output(command, shell=True, text=True)
data = result.split('\n')[1].split()
# 提取空间信息
文件系统, 总共, 已使用, 剩余 = data[0], data[1], data[2], float(data[3][:-1])
# 定义ANSI转义码
BOLD, RED, RESET = "\033[1m", "\033[31m", "\033[0m"
# 输出信息
print(f"{BOLD}文件系统: {文件系统}{RESET}")
print(f"{BOLD}总共空间: {总共}{RESET}")
print(f"{BOLD}已使用空间: {已使用}{RESET}")
# 检查剩余空间并输出相应信息
if 剩余 < 20:
    print(f"{BOLD}{RED}磁盘空间不足20GB，请及时处理！{RESET}")
else:
    print(f"磁盘空间充足，剩余空间为 {剩余} GB")

## 二、安装webui

阿里云 DSW 一键脚本v5 By [bilibili@秋葉aaaki](https://space.bilibili.com/12566101)

#### 更新日志

2023-6-11

- 修复了无法下载模型、无法启动的问题

2023-5-2

- 添加了插件，修改了插件安装方式
- 升级 torch 版本
- 更新了汉化
- 添加了部分插件（LoRA）
- 添加了下载文件用的代码

2023-5-3

- 疑似 torch2 会导致黑图，已取消

#### 1. 拉取本体

In [ ]:
# 拉取webui本体
!rm -rf /mnt/workspace/stable-diffusion-webui/
%cd  /mnt/workspace/
!git clone https://gitcode.net/overbill1683/stable-diffusion-webui
%cd stable-diffusion-webui
!mkdir repositories
%cd repositories
!git clone "https://gitcode.net/overbill1683/stablediffusion" "stable-diffusion-stability-ai"
!git clone "https://gitcode.net/overbill1683/taming-transformers" "taming-transformers"
!git clone "https://gitcode.net/overbill1683/k-diffusion" "k-diffusion"
!git clone "https://gitcode.net/overbill1683/CodeFormer" "CodeFormer"
!git clone "https://gitcode.net/overbill1683/BLIP" "BLIP"

%cd /mnt/workspace/stable-diffusion-webui
!wget -O "config.json" "https://gitcode.net/Akegarasu/sd-webui-configs/-/raw/master/config.json"

#### 2. 安装插件

In [ ]:
## 安装常用插件，填写github链接，一行一个
extension_urls = """
https://gitcode.net/ranting8323/a1111-sd-webui-tagcomplete
https://gitcode.net/overbill1683/stable-diffusion-webui-localization-zh_Hans
https://gitcode.net/ranting8323/sd-webui-additional-networks
https://gitcode.net/ranting8323/sd-webui-infinite-image-browsing
https://gitcode.net/ranting8323/stable-diffusion-webui-wd14-tagger
"""

extensions = extension_urls.strip().split("\n")

%cd /mnt/workspace/stable-diffusion-webui
for e in extensions:
    !git -C "extensions" clone {e}

#### 3. 下载模型

In [ ]:
#直链下载
def aria2(url, filename, d):
    !aria2c --console-log-level=error -c -x 16 -s 16 {url} -o {filename} -d {d}

model_url = '"https://liblibai-online.vibrou.com/models/e7e55f800035004cf0ee31d91f874a415a2b64a4.safetensors?attname=%E7%9C%9F%E5%AE%9E%E6%84%9F%E5%BF%85%E5%A4%87%E6%A8%A1%E5%9E%8B%EF%BD%9CChilloutMix_Chilloutmix-Ni-pruned-fp32-fix.safetensors"'
aria2(model_url, "chilloutmix_NiPrunedFp32Fix.safetensors", "/mnt/workspace/stable-diffusion-webui/models/Stable-diffusion")

In [ ]:
#复制oss内的模型
原始名称 = "models/chilloutmix_Ni.safetensors" #可包含父路径，最前不用加/
标准名称="model.safetensors" #双引号内填模型命名及后缀
!cp /mnt/data/"{原始名称}" /mnt/workspace/lora-scripts/sd-models/"{标准名称}" && echo 复制成功

#### 4. 下载 VAE

In [ ]:
VAE_URL = '"https://modelscope.cn/api/v1/models/dienstag/anything-v4.0/repo?Revision=master&FilePath=anything-v4.0.vae.pt"'
aria2(VAE_URL, "animevae.pt", "/mnt/workspace/stable-diffusion-webui/models/VAE")

#### 5. 下载文件（样例）

In [ ]:
# 这部分是可选的！！！！！有需要再执行！！！！！！！ 修改对应文本，运行即可下载文件
def aria2(url, filename, d):
    !aria2c --console-log-level=error -c -x 16 -s 16 {url} -o {filename} -d {d}

DOWNLOAD_URL = "文件链接" # 文件链接，需要下载的url直链才可以
FILE_NAME = "aki" # 下载保存的文件名
SAVE_DIR = "/mnt/workspace" # 文件保存的路径
aria2(DOWNLOAD_URL, FILE_NAME, SAVE_DIR)

## 三、 安装Lora-Scipts

最核心的文件的整合代码均由主要作者[秋葉aaaki](https://github.com/Akegarasu/lora-scripts)完成。
开始前**建议阅读**：
1.   [保姆式LoRA模型一键包文件](https://www.bilibili.com/video/BV1AL411q7Ub/?spm_id_from=333.337.search-card.all.click&vd_source=e6294619bd5137ea0025a3d001e715a7)
2.   [参数心得](https://www.bilibili.com/video/BV1GM411E7vk/)
3.   [训练教程](https://www.bilibili.com/video/BV1fs4y1x7p2/?spm_id_from=333.999.0.0)
4.   [阿里云部署教程](https://www.bilibili.com/video/BV1fs4y1x7p2/?spm_id_from=333.999.0.0)
>
本人只是完成阿里云下的依赖安装，使用的下载源不保证长期有效。

训练包版本为2023年10月份1.7.0秋叶最新版

以后要更新脚本直接重新执行一遍本脚本的训练包拉取，和基础依赖安装和即可更新脚本，无需更换脚本。

更新内容：使用了秋叶最新的训练包，优化了下载链接的样式，更新了大模型训练，已知著名的所有训练算法已安装。

#### 1. 拉取本体

In [ ]:
!rm -rf /mnt/workspace/lora-scripts/
%cd  /mnt/workspace/
#!git clone  https://gitee.com/taichuzhishi/lora-scripts.git #烈风包
!git clone  https://github.com/Akegarasu/lora-scripts.git #秋叶包
#!cd /mnt/workspace/lora-scripts/huggingface/ && wget -c https://liblibai-online.vibrou.com/web/model/c52a9848ee316e16b59a0fe17e17edc4c767f97f652b439d9da758d6077160ae.ckpt  -O 1.zip
#!unzip /mnt/workspace/lora-scripts/huggingface/1.zip -d /mnt/workspace/lora-scripts/huggingface/ > /dev/null 2>&1 && echo 解压成功
#!rm -r /mnt/workspace/lora-scripts/huggingface/1.zip   > /dev/null 2>&1  && echo 删除压缩包成功
!mkdir -p /mnt/workspace/lora-scripts/train/aki
!mkdir -p /mnt/workspace/lora-scripts/train/reg
!echo 训练包拉取成功

#### 2. 安装依赖

In [ ]:
#这里添加下载自改脚本
%cd  /mnt/workspace/lora-scripts/
#!wget -c https://github.com/luoyuu77/pai_dsw_sd_lora/blob/main/lora-scripts/install.bash
!bash install.bash

## 四、 启动WebUI

In [ ]:
import os
package_envs  = [
      {
        "env": "GFPGAN_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/GFPGAN.git@8d2447a2d918f8eba5a4a01463fd48e45126a379"
      },
      {
        "env": "CLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/CLIP.git@d50d76daa670286dd6cacf3bcd80b5e4823fc8e1"
      },
      {
        "env": "OPENCLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/open_clip.git@bb6e834e9c70d9c27d0dc3ecedeebeaeb1ffad6b"
      }
]
os.environ["PIP_INDEX_URL"] = "https://mirror.baidu.com/pypi/simple"
for i in package_envs:
    os.environ[i["env"]] = i["url"]

%cd /mnt/workspace/stable-diffusion-webui
!python launch.py --no-download-sd-model --xformers --share --listen --enable-insecure-extension-access

## 五、 启动Lora

In [ ]:
%cd /mnt/workspace/lora-scripts    
#跳转到UI文件
!bash run_gui.sh #界面文件点击127.0。0.0.1即可进入ui